In [38]:
#!pip install selenium

In [39]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#Passo 1 - cotação do dolar
#abrir o navegador
navegador = webdriver.Chrome()
#entrar no site
navegador.get("https://www.google.com/")

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#copiar cotação
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#Passo 2 - cotação do euro
navegador = webdriver.Chrome()
navegador.get("https://www.google.com/")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#copiar cotação
cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#Passo 3 - cotação do ouro
navegador = webdriver.Chrome()
navegador.get("https://www.melhorcambio.com/ouro-hoje")
#copiar cotação
#replace subistitui 
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')

#fechar o navegador
navegador.quit()

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [40]:
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
tabela

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [41]:
#tabela.loc[linha,coluna]
tabela.loc[tabela['Moeda']=='Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda']=='Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda']=='Ouro', 'Cotação'] = float(cotacao_ouro)
tabela


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.172800,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.290067,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.172800,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.172800,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.290067,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.172800,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,293.370000,7000.00,1.15,8050.000


In [42]:
# Preço de Compra = Preço Original * Cotação
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

# Preço de Venda = Preço de Compra * Margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']
tabela

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.172800,5172.748272,1.40,7241.847581
1,Carro Renault,4500.00,Euro,5.290067,23805.303192,2.00,47610.606384
2,Notebook Dell,899.99,Dólar,5.172800,4655.468272,1.70,7914.296062
3,IPhone,799.00,Dólar,5.172800,4133.067200,1.70,7026.214240
4,Carro Fiat,3000.00,Euro,5.290067,15870.202128,1.90,30153.384043
5,Celular Xiaomi,480.48,Dólar,5.172800,2485.426944,2.00,4970.853888
6,Joia 20g,20.00,Ouro,293.370000,5867.400000,1.15,6747.510000


### Agora vamos exportar a nova base de preços atualizada

In [43]:
tabela.to_excel('Produtos novo.xlsx', index = False)